# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [2]:
# Install the correct packages
!python -m pip install -qU jq lark langchain langchain-community langchain-openai tiktoken elasticsearch

# Correct imports
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass


## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial. 

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [8]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="el_search",
    embedding=embeddings,
)

C:\Users\kenneth\AppData\Local\Temp\ipykernel_9244\2301446582.py:12: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use :class:`~Use class in langchain-elasticsearch package` instead.
  vectorstore = ElasticsearchStore(


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [9]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [19]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from datetime import datetime

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["name"] = record.get("name", "")
    metadata["summary"] = record.get("summary", "")
    metadata["url"] = record.get("url", "")
    metadata["category"] = record.get("category", "")
    
    # Fix updated_at
    updated_raw = record.get("updated_at", "")
    try:
        metadata["updated_at"] = datetime.fromisoformat(updated_raw).isoformat()
    except Exception:
        metadata.pop("updated_at", None)  # Or set to None if invalid
    
    return metadata


# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,
    chunk_overlap=400,
)
docs = loader.load_and_split(text_splitter=text_splitter)

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [20]:
from datetime import datetime

def clean_metadata(record, metadata):
    if "updated_at" in record:
        try:
            # Example: Convert to ISO format
            metadata["updated_at"] = datetime.fromisoformat(record["updated_at"]).isoformat()
        except Exception:
            metadata["updated_at"] = None  # or remove if invalid
    return metadata


In [21]:
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=clean_metadata,
)


In [22]:
for d in docs[:3]:
    print(d.metadata)


{'source': 'C:\\Users\\kenneth\\Desktop\\IronHack AI Engineer\\multi-query-retriever\\lab-chatbot-with-multi-query-retriever\\temp.json', 'seq_num': 1, 'name': 'Work From Home Policy', 'summary': 'This policy outlines the guidelines for full-time remote work, including eligibility, equipment and resources, workspace requirements, communication expectations, performance expectations, time tracking and overtime, confidentiality and data security, health and well-being, and policy reviews and updates. Employees are encouraged to direct any questions or concerns', 'url': './sharepoint/Work from home policy.txt', 'category': 'teams', 'updated_at': '2020-03-01T00:00:00'}
{'source': 'C:\\Users\\kenneth\\Desktop\\IronHack AI Engineer\\multi-query-retriever\\lab-chatbot-with-multi-query-retriever\\temp.json', 'seq_num': 2, 'name': 'April Work From Home Update', 'summary': 'Starting May 2022, employees will need to work two days a week in the office. Coordinate with your supervisor and HR depart

In [23]:
vectorstore.from_documents(
    docs,
    embeddings,
    index_name="el_search",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)


llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [27]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """"You are an assistant for question-answering tasks. First, verify whether the context contains information that answers the question. If it does, answer in full. If not, say 'I don't know based on the context.'"
 
    
    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The North America South America region (NASA) has two Area Vice-Presidents: Laura Martinez is the Area Vice-President of North America, and Gary Johnson is the Area Vice-President of South America.


In [28]:
ans = chain.invoke("Who oversees operations in the European region?")
print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. Who is responsible for managing operations in the European region?', '2. What department is in charge of overseeing operations in Europe?', '3. Which individual or team is responsible for the operations in the European region?']


---- Answer ----

The Area Vice-President for Europe, Rajesh Patel, oversees operations in the European region.


In [29]:
ans = chain.invoke("What does the Banana Innovation Department focus on?")
print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the main areas of focus for the Banana Innovation Department?', '2. Can you provide more information on the specific focus of the Banana Innovation Department?', '3. How does the Banana Innovation Department prioritize their focus areas?']


---- Answer ----

I don't know based on the context.


**My reflections**

Through experimenting with different questions and prompt techniques, the differences in the answered outputs where quite different.

Using the original Context prompt, the last question I asked regarding the Banana Innovation Department, generated the following response:

"The Banana Innovation Department focuses on developing and launching innovative products and services in emerging technology areas, as well as enhancing post-sales support and customer service to improve customer satisfaction. This is outlined in the Product/Service Portfolio focus area of the sales strategy document for fiscal year 2024 it gave this answer."

This means that it started to hallucinate this, and did not adhere strictly to the rule of not using anything outside of context. I adjusted the prompt to be very strict with only using the source material and it produced a correct output of saying: "I don't know based on the context."

**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?